# Portfolio Performance Analysis

This notebook analyzes daily portfolio performance using Polars.

In [ ]:
import os
import polars as pl
import numpy as np
import matplotlib.pyplot as plt


## 1. Load Data


In [ ]:
datasets_dir = os.path.abspath(os.path.join('..', 'datasets'))
tx = pl.read_csv(f'{datasets_dir}/cleaned_transactions.csv')
assets = pl.read_parquet(f'{datasets_dir}/assets.parquet')


## 2. Join and Calculate Trade Value


In [ ]:
df = tx.join(assets, on='asset_id', how='inner')
df = df.with_columns((pl.col('amount') * pl.col('price')).alias('trade_value'))


## 3. Date Handling
Ensure date is parsed and extract trade_date.

In [ ]:
if df['date'].dtype != pl.Datetime:
    df = df.with_columns(pl.col('date').str.strptime(pl.Datetime, format='%Y-%m-%dT%H:%M:%S%.f').alias('date'))
df = df.with_columns(pl.col('date').dt.date().alias('trade_date'))


## 4. Aggregate Daily Performance


In [ ]:
daily_perf = (
    df.group_by('trade_date')
      .agg([pl.sum('trade_value').alias('daily_trade_value'), pl.count().alias('num_trades')])
      .sort('trade_date')
)
print('Daily performance sample:')
print(daily_perf.head())


## 5. Cumulative and Daily Returns


In [ ]:
daily_perf = daily_perf.with_columns([pl.cum_sum('daily_trade_value').alias('cumulative_value')])
daily_perf = daily_perf.with_columns([(pl.col('daily_trade_value') / pl.col('cumulative_value').shift(1) * 100).alias('daily_return_pct')])


## 6. Performance Metrics


In [ ]:
avg_daily_return = daily_perf['daily_return_pct'].mean()
std_daily_return = daily_perf['daily_return_pct'].std()
sharpe = avg_daily_return / std_daily_return * np.sqrt(252)
print(f'\nPortfolio Performance Metrics:')
print(f'Average Daily Return: {avg_daily_return:.2f}%')
print(f'Daily Volatility: {std_daily_return:.2f}%')
print(f'Annualized Sharpe Ratio: {sharpe:.2f}')


## 7. Save Results


In [ ]:
daily_perf.write_csv(f'{datasets_dir}/portfolio_performance.csv')
print('Saved portfolio performance to CSV.')
